<a href="https://drive.google.com/file/d/1LavEDKL8j1XDyVLBJ8JyTABinqqIQweV/view?usp=sharing" target="_blank" >
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>

In [ ]:
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
#Importing libraries
import numpy as np
import seaborn as sns
import tensorflow as tf
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from keras.layers import LeakyReLU
from tqdm.auto import tqdm

2024-08-21 16:10:38.310291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-21 16:10:38.320360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-21 16:10:38.323441: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-21 16:10:38.331677: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-21 16:10:38.863308: W tensorflow/compiler/tf2

In [ ]:
# Defining necessary parameters
batch_size = 512
latent_dim = 8

## Define a Discriminator Model

In [ ]:
# Build a discriminator neural network
def build_discriminator(dim):
  model = Sequential()
  for _ in range(2):
    model.add(____) # Dense layer with 64 neurons and activation = LeakyReLU
  model.add(Dense(___, activation=____))
  return ____

## Define a Generator Model

In [ ]:
# Build a generator neural network
def build_generator(latent_dim, output_dim):
  model = Sequential()
  for _ in range(4):
    model.add(____) # Dense layer with 16 neurons and activation = LeakyReLU
  model.add(Dense(output_dim))
  return ____

## Defining and Training the GAN


In [ ]:
# Generate random uniform noise to input to the generator
def generate_input_noise(batch_size, latent_dim):
    return ____

In [ ]:
#  Generate real data from a normal distribution to train discriminator
def get_real_data(n_samples,output_dim):
    ____ # Set the tensorflow random seed as 109
    return ____

In [ ]:
# Build the GAN
g_model = build_generator(____, ___)
d_model = build_discriminator(___)

## Losses or Optimizers

In [ ]:
bce_loss = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# Discriminator Loss
def discriminator_loss(real_output, fake_output):
    real_loss = bce_loss(tf.ones_like(real_output), real_output)
    fake_loss = bce_loss(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(d_predictions):
    return bce_loss(tf.ones_like(d_predictions), d_predictions)

In [ ]:
d_optimizer = ____ # Adam optimizer with learning rate = 0.0001
g_optimizer = ____ # Adam optimizer with learning rate = 0.006

## Training

Define your training loop by iterating through the number of epochs and the batches of real data generated before.

Remember, the training loop begins with generator receiving 100-D noise as input. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.

Define `train_step` function that trains the generator and discriminator over a batch

In [ ]:
# Compiles the train_step function into a callable TensorFlow graph
# Also speeds up the training time
@tf.function
def train_step():

    real_images = get_real_data(batch_size // 2, 1)
    generated_images = g_model(generate_input_noise(batch_size // 2, latent_dim))

    # Train the discriminator.
    with tf.GradientTape() as tape:
        pred_fake = d_model(generated_images)
        pred_real = d_model(real_images)

        d_loss = discriminator_loss(pred_real, pred_fake)

    grads = tape.gradient(d_loss, d_model.trainable_variables)
    d_optimizer.apply_gradients(zip(grads, d_model.trainable_variables))

    #-----------------------------------------------------------------#

    # Sample random points in the latent space.
    random_latent_vectors = generate_input_noise(batch_size, latent_dim)

    # Train the generator (note that we should *not* update the weights
    # of the discriminator)!
    with tf.GradientTape() as tape:
        fake_images = g_model(random_latent_vectors)
        predictions = d_model(fake_images)
        g_loss = generator_loss(predictions)

    grads = tape.gradient(g_loss, g_model.trainable_variables)
    g_optimizer.apply_gradients(zip(grads, g_model.trainable_variables))

    return d_loss, g_loss

In [ ]:
D_loss = []
G_loss = []
G_predict=[]
epochs = 1000

for epoch in tqdm(range(epochs)):
    tf.random.set_seed(109+epoch)
    d_loss, g_loss = _____
    D_loss.append(____)
    G_loss.append(____)

    test_noise = generate_input_noise(10000, latent_dim)
    fake_samples = g_model.predict(____)
    G_predict.append(____)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
plt.close(fig)
def animate(i):
  ax[1].cla()
  # Plot loss and accuracy
  ax[0].plot(np.arange(4*i), G_loss[0:4*i],label='G loss',c='darkred',zorder=50,alpha=0.8)
  ax[0].plot(np.arange(4*i), D_loss[0:4*i],label='D loss',c='darkblue',zorder=55,alpha=0.8)
  ax[0].set_xlim(-5, epochs+5)
  ax[0].set_ylim(-0.05, 1.55)
  ax[0].set_xlabel('Epoch')

  #Plot distributions
  x_vals = np.linspace(-3, 3, 301)
  y_vals = stats.norm(0,1).pdf(x_vals)
  ax[1].plot(x_vals, y_vals, color='blue', label='real')
  ax[1].fill_between(x_vals, np.zeros(len(x_vals)), y_vals, color='blue', alpha=0.6)
  a = sns.kdeplot(G_predict[4*i].flatten(), color='red', alpha=0.6, label='GAN', ax=ax[1], shade=True)
  ax[1].set_xlim(-3, 3)
  ax[1].set_ylim(0, 0.82)
  ax[1].set_xlabel('Sample Space')
  ax[1].set_ylabel('Probability Density')

simulation = FuncAnimation(fig, animate, frames=epochs//4, interval=100, repeat=True)
HTML(simulation.to_html5_video())